In [1]:
from config.configuration import RunDetails

# runDetail = RunDetails('config.yml', 'ASVspoof-2019_training')
# runDetail = RunDetails('config-mfcc.yml', 'ASVspoof-2019_training_mfcc')
# runDetail = RunDetails('config.yml', 'ASVspoof-2019_small-eval-1')
runDetail = RunDetails('config.yml', 'Compare-Sksmta-training')

notebookName = 'audio-deepfake-detection-training'

In [2]:
configFilename = runDetail.configFilename
runJobId = runDetail.jobId

In [ ]:
import json

import config.configuration as configuration
import model_definitions.model_cnn_definition as model_cnn_definition
from notebook_utils import notebookToPython
from preprocessors.abstract_preprocessor import AbstractPreprocessor
from preprocessors.preprocessor_factory import PreprocessorFactory
from processors.basic_model_training_processor import BasicModelTrainingProcessor
from processors.basic_model_evaluation_processor import BasicModelEvaluationProcessor

In [ ]:
config = configuration.ConfigLoader(configFilename)

notebookToPython(notebookName)
job = config.getJobConfig(runJobId)

prettyJson = json.dumps(job.__dict__, indent=4)
print(f"job: {prettyJson}")

if (job.newModelGenerated == False):
    raise ValueError("This notebook is meant for training. Select a job without a value for 'persisted-model' set.")

Write python file
Generating new model name: output/Compare-Sksmta-training_2025-04-06T21-07-25.252600.libjob
Assigned model name: output/Compare-Sksmta-training_2025-04-06T21-07-25.252600.libjob


In [5]:
preproc_factory = PreprocessorFactory()
preprocessor: AbstractPreprocessor = preproc_factory.newPreprocessor(job.preprocessor)


MelSpectrogramPreprocessor


In [6]:
X, y_encoded, true_labels = preprocessor.extract_features_jobSource(job, job.dataPathSuffix)

Loading C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval-small.trl.txt...
fullDataPath: C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_eval_small/flac
Loading audio files: 1
Loading audio files: 2
Loading audio files: 3
Loading audio files: 4
Number of audio files loaded: 4


In [7]:
trainingProc = BasicModelTrainingProcessor(job, model_cnn_definition.ModelCnnDefinition)
model, X_train, X_test, y_train, y_test = trainingProc.process(X, y_encoded, 1)

useTrainingSplitRandomState: -1
IMPORTANT: Splitting using the same method as in "Sksmta" base code
Training using 3 files.
    def buildModel(self) -> Model:
        print(f"__job__: {self.__job__}")
        input_shape = (self.__job__.numMels, self.width, self.channels)
        model_input = Input(shape = input_shape)
        job = self.__job__

        # Derived from:
        # Anagha, R., Arya, A., Narayan, V. H., Abhishek, S., & Anjali, T. (2023).
        #   Audio Deepfake Detection Using Deep Learning.
        #   2023 12th International Conference on System Modeling & Advancement in Research Trends (SMART), System Modeling & Advancement in Research Trends (SMART), 2023 12th International Conference On, 176–181.
        #   https://doi.org/10.1109/SMART59791.2023.10428163
        x = Conv2D(filters=32, kernel_size=job.kernelSize, activation='relu')(model_input)
        x = MaxPooling2D(pool_size=job.poolSize)(x)
        x = Conv2D(filters=64, kernel_size=job.kernelSize, activati

### Test Model

In [8]:
from processors.model_evaluation_result import ModelEvaluationResult


evaluationProc = BasicModelEvaluationProcessor(job, model)
results: ModelEvaluationResult = evaluationProc.process(X_test, y_test, true_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
  Batches: 1 - Files: 1 - Score: 0.0 - Elements: 1


In [9]:
from postprocessors.confusion_matrix_plot import ConfusionMatrixDetails, ConfusionMatrixPlot
import json

print(f'results.test: {results.test}')
print(f'results.pred: {results.pred}')
print(f'classes: {job.classes}')
print(f'y_encoded: {y_encoded}')
# confusionMatrixPlot = ConfusionMatrixPlot()
# cmDetails: ConfusionMatrixDetails = confusionMatrixPlot.plot(job.classes, results.test, results.pred)


results.test: [[0. 1.]]
results.pred: [0]
classes: ['spoof', 'bonafide']
y_encoded: [[1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [10]:
print("\n")
report = evaluationProc.reportSnapshot(trainingProc)
evaluationProc.writeReportToFile(job.persistedModelResults, report)
print(f'Results:\n{report}')



Results:
---- Training (start) ----
start time: 2025-04-07T02:07:29.427804+00:00
end time: 2025-04-07T02:07:33.920937+00:00
elapsed: 0:00:04.493133

model file: output/Compare-Sksmta-training_2025-04-06T21-07-25.252600.libjob
batch count: 0
file count: 0
job: {
    "jobId": "Compare-Sksmta-training",
    "inputFileBatchSize": 1000,
    "outputFolder": "output",
    "dataPathRootRaw": "$HOMEDRIVE$HOMEPATH/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathRoot": "C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathSuffix": "LA/ASVspoof2019_LA_eval_small/flac",
    "dataExtension": ".flac",
    "trainingSplitRandomState": -1,
    "labelFilename": "LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval-small.trl.txt",
    "executeToCategoricalForLabels": true,
    "classes": [
        "spoof",
        "bonafide"
    ],
    "numClasses": 2,
    "sampleRate": 16000,
    "duration": 5,
    "numMels": 128,
    "maxTimeSteps": 109,
    "kernelSize": [
        2,
        2
